In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as  transforms
from torchvision import datasets

In [3]:
transform = transforms.Compose([
                      transforms.Grayscale(num_output_channels=1),
                      transforms.RandomHorizontalFlip(),
                      transforms.Resize((28,28)),
                      transforms.ToTensor(),
                      transforms.Normalize((0.5,),(0.5,))
                      ])

In [23]:
# Path to the dataset (update the path as needed)
data_dir = "C:/Swetha/ASEB/Infosys_internship/brain_tumor_dataset"

# Load the dataset with transformations
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Paths to the pre-split datasets (update the paths as needed)
train_dir = "C:/Swetha/ASEB/Infosys_internship/brain_tumor_dataset/Training"
test_dir = "C:/Swetha/ASEB/Infosys_internship/brain_tumor_dataset/Testing"


# Load the pre-split datasets with transformations
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# Create DataLoaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

images, labels = next(iter(train_loader))
print("Image size:", images.shape) 


Image size: torch.Size([32, 1, 28, 28])


In [24]:
class MLP(nn.Module):
  def __init__(self):
    super(MLP,self).__init__()
    self.fc1 = nn.Linear(28*28,128)
    self.fc2 = nn.Linear(128,4)
    self.relu=nn.ReLU()

  def forward(self,x):
    x = x.view(-1,28*28)
    x = self.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [25]:
model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer =optim.Adam(model.parameters(),lr=0.001)

In [27]:
epochs = 10
for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  correct =0
  total =0
  for inputs, labels in train_loader:
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs ,labels )
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    _,predicted = torch.max(outputs,1)
    total +=labels.size(0)
    correct +=(predicted ==labels).sum().item()

  print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)} , Accuracy: {100*correct/total}")

Epoch 1/10, Loss: 0.5601346735847729 , Accuracy: 79.39425770308124
Epoch 2/10, Loss: 0.5363769977452368 , Accuracy: 79.74439775910363
Epoch 3/10, Loss: 0.5186508834029043 , Accuracy: 80.89985994397759
Epoch 4/10, Loss: 0.49688637980868694 , Accuracy: 81.53011204481793
Epoch 5/10, Loss: 0.4848660554466301 , Accuracy: 82.28291316526611
Epoch 6/10, Loss: 0.4709683665017176 , Accuracy: 82.44047619047619
Epoch 7/10, Loss: 0.45650598090454186 , Accuracy: 83.17577030812325
Epoch 8/10, Loss: 0.42984824923163684 , Accuracy: 83.84103641456582
Epoch 9/10, Loss: 0.4223604968140245 , Accuracy: 83.98109243697479
Epoch 10/10, Loss: 0.4226997289910663 , Accuracy: 83.66596638655462


In [ ]:
#  Evaluate on Test Data
model.eval()
correct =0
total =0
act_labels =torch.tensor([])
model_predicted =torch.tensor([])

with torch.no_grad():
  for inputs, labels in test_loader:
    outputs =model(inputs)
    act_labels = torch.cat((act_labels ,labels ) ,dim =0)
    _, predicted = torch.max(outputs,1)
    model_predicted = torch.cat((model_predicted ,predicted),dim=0 )
    total +=labels.size(0)
    correct += (predicted ==labels).sum().item()
print(f"Test Accuracy: {100*correct/total:.2f}")